# Advanced inheritance

**Outline**:
- Reminder on inheritance
- Proper multiple inheritance
- Metaclasses
- Composition over inheritance
- Dynamic object creation

TODO:
- `super(X, y)` 
- `__init_subclass__`
- 


## Reminder on inheritance

### Inheritance basics
- ABC/ABCMeta, abstractmethod, super

### Multiple inheritance
- Diamond problem

## Proper multiple inheritance

### Pitfalls
The MRO ensures a clear ordering of the class hierarchy, with good properties and deterministic behavior. It does not solve all the problem however. Consider the following piece of code.

In [ ]:
class Printer:
    def output(self):
        return "Printing..."

class Logger:
    def output(self):
        return "Logging..."

class Service(Printer, Logger):
    pass

s = Service()
print(s.output())


In this case, we have a collision between the `output` methods, and one masks the other. This happens with common names, in both methods and attributes. A special case is dunder methods (where the name is imposed), with the most frequent issue being  `__init__`.

In [ ]:
class A:
    def __init__(self):
        print("Initializing A")

class B:
    def __init__(self):
        print("Initializing B")
        
class C(A, B):
    def __init__(self):
        super().__init__()
        print("Initializing C")

C()  # Skipping B

Note that you can partially circumvent the above issue with more specific calls, but this then breaks the MRO properties, leading to other issues

In [ ]:
class A:
    def __init__(self):
        print("Initializing A")

class B(A):
    def __init__(self):
        print("Initializing B")
        super().__init__()

class C(A):
    def __init__(self):
        print("Initializing C")
        super().__init__()

class D(B, C):
    def __init__(self):
        print("Initializing D")
        B.__init__(self) # Explicit choice of super class
        C.__init__(self) # Explicit choice of super class 

_ = D()  # C and A are initialized twice

The double initilization might just be a waste of time, but it might also be an issue in case of side effects.

The remainder of this section is about best practices in the context of multiple inheritance.

### Interface/protocol

In OOP, a common way of doing "soft" multiple inheritance is to have *interfaces*. An interface is like an API, it exposes some method that must be implemented but does not provide an implementation. It is a contract to the user saying that a concrete class will have this/those methods implemented. Typically, an object can implement several interfaces without risk name collision since no such "inheritances" provide a real implementation to mask.

Prior to typing, this was not widely used in Python, where the philosophy was Duck typing (try instead of check). Typing introduce the need for a `Protocol` mechanism, which can also be used to define something similar to an interface.


In [ ]:
from typing import Protocol, runtime_checkable


@runtime_checkable  # To be able to use isinstance at runtime
class SupportsFlush(Protocol):
    def flush(self) -> None: ...


class Serializer(SupportsFlush):
    def flush(self) -> None:
        print("Flushing data...")


When used a priori to build a class hierarchy, we inherit from the `Protocol` child class. `Protocol` can be used at posteriori as well without being part of the class hierarchy.

### Mixin

A common way to use multiple inheritance is to use the Mixin construct. A Mixin is a piece of code designed that encapsulate a common behavior, allowing the share the implementation between class that are not part of a same hierarchy. 

Here is an example:

In [ ]:
class ClonableMixin:
    def clone(self, deep: bool = False):
        import copy
        return copy.deepcopy(self) if deep else copy.copy(self)
    
class ReprMixin: 
    # Only valid for custom dataclass-like classes
    def __repr__(self):
        return (
            f"{self.__class__.__name__}("
            + ", ".join(f"{k}={v!r}" for k, v in self.__dict__.items())
            + ")"
        )
    
class Shape:
    def area(self) -> float:
        raise NotImplementedError()
    
    
class Rectangle(ClonableMixin, ReprMixin, Shape):
    def __init__(self, width: float, height: float):
        self.width = width
        self.height = height

    def area(self) -> float:
        return self.width * self.height
    
class Person(ReprMixin, ClonableMixin):  # Reusable
    def __init__(self, name):
        self.name = name

Rectangle(10, 20).clone()

Mixin is a powerful construct, provided some good practices are observed:
- a mixin scope should be as small as possible (one mixin = one purpose), prefer chaining orthogonal scopes;
- suffix the name with `Mixin` to indicate a clear purpose;
- no `__init__` (avoid issues highlighted above), as little state as possible;
- always place the mixins first in the MRO (why :question:);
- don't sublcass mixins;
- avoid when non-trivial;
- unit-test mixins in isolation and perform integration tests over the full classes.

> Although mixins should avoid statefulness and initialization, there are ways to circumvent that (https://realpython.com/python-mixin/#how-can-you-use-stateful-mixins-safely). Whether this is a good idea is not that clear.

### Cooperation
- cooperative `__init__`
- chainable hooks (cooperative super)
- mergeable result (cooperative super)

Another way to take advantage of multiple inheritance is to create a collection of classes in a cooperative manner.

#### Cooperative initialization
One way to make the initialization work in complex hierarchy is to swallow non-relevant arguments in kwargs:

In [33]:
class Swimmer:
    def __init__(self, swim_speed: float, **kwargs):
        self.swim_speed = swim_speed
        super().__init__(**kwargs)

class Flyer:
    def __init__(self, fly_speed: float, **kwargs):
        self.fly_speed = fly_speed
        super().__init__(**kwargs)

class FlyingFish(Swimmer, Flyer):
    def __init__(self, swim_speed: float, fly_speed: float):
        super().__init__(swim_speed=swim_speed, fly_speed=fly_speed)

fish = FlyingFish(swim_speed=10, fly_speed=20)
print(f"Swim speed: {fish.swim_speed}, fly speed: {fish.fly_speed}")

Swim speed: 10, fly speed: 20


Note in the example above how the the `Swimmer` and `Flyer` can be inverted, since the `super` call will pass on the call of initialization in the MRO.

#### Cooperative returns (merging)

Another way to design class copperatively, is to be able to merge results:

In [34]:
class Base:
    def validate(self):
        return []

class NameValidation:
    def validate(self):
        errors = super().validate()
        if not getattr(self, "name", None):
            errors.append("Missing name")
        return errors

class AgeValidation:
    def validate(self):
        errors = super().validate()
        if getattr(self, "age", 0) < 0:
            errors.append("Age invalid")
        return errors

class PermissionValidation:
    def validate(self):
        errors = super().validate()
        if not getattr(self, "is_admin", False):
            errors.append("Not an admin")
        return errors

class User(NameValidation, AgeValidation, PermissionValidation, Base):
    def __init__(self, name, age, is_admin):
        self.name = name
        self.age = age
        self.is_admin = is_admin


user = User(name="", age=-1, is_admin=False)
print(user.validate())

['Not an admin', 'Age invalid', 'Missing name']


Note how the `Base` is still included; that is where chained calls end up and create the empty list which gets filled up with the other calls. This pattern can be used with dictionaries, or as part of more complex design patterns (builder, visitor). 

> The validation subclasses can seen as Mixins.

#### Cooperative hooks

Another form of cooperation is to propose overriable hooks:

In [38]:
class Pipeline:
    def run(self, data):
        data = self.pre(data)
        data = self.process(data)
        data = self.post(data)
        return data

    # Noop defaults to end up the MRO
    def pre(self, d): return d
    def process(self, d): return d
    def post(self, d): return d

class LoggingMixin:
    def pre(self, d):
        print("pre >>")
        return super().pre(d)
    
    def post(self, d):
        print("<< post")
        return super().post(d)

class Upper(Pipeline):
    def process(self, d): 
        print("<upperizing>")
        return d.upper()

class Job(LoggingMixin, Upper): pass


job = Job()
print(job.run("hello"))  # pre, post, HELLO

pre >>
<upperizing>
<< post
HELLO


## Composition over inheritance

One common piece of advice is that composition should be prefered over inheritance. Here is an example on how to structure the validation example of above with composition:



In [41]:
from abc import ABC, abstractmethod
from typing import Collection


class Validator(ABC):
    @abstractmethod
    def validate(self, obj) -> Collection[str]:
        raise NotImplementedError()
    


class CompositeValidator:
    def __init__(self, *validators: Validator):
        self._validators = validators
    
    def validate(self, obj) -> Collection[str]:
        errors = []
        for v in self._validators:
            errors.extend(v.validate(obj))
        return errors
    
class NameValidator(Validator):
    def validate(self, obj) -> Collection[str]:
        if not getattr(obj, "name", None):
            return ["Missing name"]
        return []
    
class AgeValidator(Validator):
    def validate(self, obj) -> Collection[str]:
        if getattr(obj, "age", 0) < 0:
            return ["Age invalid"]
        return []
    
class PermissionValidator(Validator):
    def validate(self, obj) -> Collection[str]:
        if not getattr(obj, "is_admin", False):
            return ["Not an admin"]
        return []
    
class User:
    _validator = CompositeValidator(
        NameValidator(),
        AgeValidator(),
        PermissionValidator(),
    )
    
    def validate(self):
        return self._validator.validate(self)
    
    def __init__(self, name, age, is_admin):
        self.name = name
        self.age = age
        self.is_admin = is_admin

user = User(name="", age=-1, is_admin=False)
print(user.validate())

['Missing name', 'Age invalid', 'Not an admin']


:wrench: convert the swimmer, flyer, flyingfish case to a compositional pattern

In [ ]:
# Swim, fly, fish

| Aspect                          | Composition                                                                                                    | Inheritance                                                                |
| ------------------------------- | -------------------------------------------------------------------------------------------------------------- | -------------------------------------------------------------------------- |
| **Reusability**                 | Components can be reused across unrelated classes easily (e.g., a `Logger` can be injected into many classes). ✅ | Reuse tied to hierarchy → only subclasses benefit (but Mixin and cooperation)                         |
| **Flexibility**                 | Swap components at runtime (different strategies, mocks in tests). Loose coupling ✅                             | Class behavior fixed at class-definition time.  Tight coupling            |
| **Hierarchy depth**             | Flat structures; avoids fragile deep trees. ✅                                                                   | Encourages deep hierarchies that are harder to maintain.                   |
| **Name collision**              | Components live in their own namespace, fewer accidental overrides. ✅                                           | Method/attribute collision possible (diamond problem).            |
| **Evolution**                   | Easy to extend by adding/removing components ✅                                                                  | Extending requires modifying hierarchy; can lead to brittle base classes.  |
| **Testing**                     | Components can be tested in isolation. ✅                                                                        | Harder to test base classes without concrete subclasses.                   |
| **Runtime behavior**            | Behavior can be delegated dynamically (Strategy, State patterns). ✅                                             | Behavior fixed by parent methods unless overridden.                        |
| **Discoverability**             | Can obscure where a method is defined (delegated via `__getattr__`, forwarding).                                 | Hierarchy shows all inherited methods; IDEs handle it well (although investigating several path is a cognitive challenge). ✅               |
| **Boilerplate**                 | Requires explicit delegation/wrappers (`self.comp.method()`).                                                    | Inherited methods work “for free” once defined. ✅                           |
| **Performance**                 | Slight overhead for delegation calls.                                                                            | Direct method lookup is slightly faster. ✅                                  |
| **Simplicity (small projects)** | Might feel verbose for trivial extensions.                                                                       | Inheritance can be simpler when just adding tiny customizations. ✅          |

In short, composition offers more flexibility and better utility but is more verbose by needing to setup properly the indirection layer. Inheritance patterns work best when the full scope is known appriori and we can commit to the design.

## Metaclasses

A metaclass creates class in the same way a class create instances. Metaclass offers a mechanism to customize a hierarchy of classes. The metaclass can be used to control how class are defined and how instances are created.

:warning: Since it is a class creation mechanism and not an instance creation, the `__new__`/`__init__` are called at the subclass creation (when it is read from the REPL). The `__call__` is called when a instance is created.

See the example below.

In [55]:
class Singleton(type):
    _instances = {}
    
    def __call__(cls, *args, **kwargs):
        if cls not in cls._instances:
            print(f"Creating instance of {cls.__name__} with args={args}, kwargs={kwargs}")
            instance = super().__call__(*args, **kwargs)
            cls._instances[cls] = instance
        return cls._instances[cls]
    

class Logger(metaclass=Singleton):
    def log(self, msg):
        print(f"[LOG] {msg}")


logger1 = Logger()
print("---")
logger2 = Logger() 
print("---")
print(logger1 is logger2)  # True, same instance

Creating instance of Logger with args=(), kwargs={}
---
---
True


In [ ]:
class Cloneable(type):
    def __new__(cls, name, bases, attrs):
        print(f"Defining class {name} with Cloneable metaclass")
        def clone(self, deep: bool = False):
            import copy
            return copy.deepcopy(self) if deep else copy.copy(self)
        attrs['clone'] = clone
        return super().__new__(cls, name, bases, attrs)
    
print("---")
class Person(metaclass=Cloneable):
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Person(name={self.name!r}) @ {id(self)}"
print("---")

alice = Person("Alice")
print(alice)
print(alice.clone())
    

---
Defining class Person with Cloneable metaclass
---
Person(name='Alice') @ 140109912331264
Person(name='Alice') @ 140109912333856


:wrench: 

Metaclass is a powerful mechanism, which is seldom used for a couple of reasons. Firstly, it adds complexity in an unsual way. More importantly, a class can only one metaclass, so combining metaclasses is not possible. In case of multiple inhertiance with different (incompatible) metaclasses, Python will raise a `TypeError`, making it hard to work in context where several metaclasses co-exists. As a consequence, compositional patterns are to be preferred, like class decoration:

In [ ]:
def cloneable(cls):
    def clone(self, deep: bool = False):
        import copy
        return copy.deepcopy(self) if deep else copy.copy(self)
    cls.clone = clone
    return cls

@cloneable
class Person:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return f"Person(name={self.name!r}) @ {id(self)}"
    
class Person2(Person): pass

alice = Person("Alice")
print(alice)
print(alice.clone())

Person2("Bob").clone()  # Works as well

Person(name='Alice') @ 140109912035776
Person(name='Alice') @ 140109912035440


Person(name='Bob') @ 140109912037168

Hence the `cloneable` decorator is re-usable and does not interact with class hierarchy. 

:question: How come the clone method is inherited as well?

## Dynamic object creation ?

## Closing words
